<a href="https://colab.research.google.com/github/urieliram/tc_uc/blob/main/EL_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving dirdat.rar to dirdat.rar
User uploaded file "dirdat.rar" with length 6364850 bytes


In [2]:
!pip install unrar
!unrar x "/content/dirdat.rar"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/dirdat.rar

Creating    dirdat                                                    OK
Extracting  dirdat/ANGVOLNODAL_1.csv                                       0%  OK 
Extracting  dirdat/AREAMEM.csv                                             0%  OK 
Extracting  dirdat/ARRARC.csv                                              0%  OK 
Extracting  dirdat/ARRARD.csv                                              0%  OK 
Extracting  dirdat/ASIGNH.csv                                              0%  OK 
Extracting  dirdat/ASIGNRC.csv                                             0%  OK 
Extracting  dirdat/ASIGNRD.csv                                             0%  OK 
Extracting  dirdat/ASIGNRE.csv                                            

In [ ]:
!unzip "dirdat_2_sept-20220608T234128Z-001.zip" -d "/content"

unzip:  cannot find or open dirdat_2_sept-20220608T234128Z-001.zip, dirdat_2_sept-20220608T234128Z-001.zip.zip or dirdat_2_sept-20220608T234128Z-001.zip.ZIP.


In [ ]:
from os import listdir
from os.path import isfile, join
import numpy as np

##Testing

In [ ]:
def validateFiles():
  f = open("/content/catalogo.txt","r")
  validFiles = [i.replace("\n","") for i in f]
  f.close()
  onlyfiles = [f.split(".")[0] for f in listdir("/content/output") if isfile(join("/content/output", f))]
  for i in validFiles:
    if i not in onlyfiles:
      return False
  return True

In [ ]:
def docSumByColum(fileName):
  data = np.transpose(np.genfromtxt('/content/output/{}.csv'.format(fileName), delimiter=','))
  return [sum(i) for i in data][:-1]

In [ ]:
def extractFromFile(generator,file,method="col",col=None):
  if method == "min":
    return min(np.genfromtxt("/content/output/{}.csv".format(file), delimiter=',')[generator])
  if method == "max":
    return max(np.genfromtxt("/content/output/{}.csv".format(file), delimiter=',')[generator])
  if method == "col":
    return np.genfromtxt("/content/output/{}.csv".format(file), delimiter=',')[generator][col]

In [ ]:
def extractFromFileByRange(generator,file,rang,pos1=0,pos2=1):
  data = np.genfromtxt("/content/output/{}.csv".format(file), delimiter=',')
  lag = [0]
  cost = [0]
  for i,j in enumerate(data[generator*rang:(generator*rang)+rang]):
    if j[pos1] != 0 and j[pos2] != 0:
      if i == 0:
        lag[0] = j[pos1]
        cost[0] = j[pos2]
      else:
        lag.append(lag[-1]+j[pos1])
        cost.append(j[pos2])
  return lag,cost

In [ ]:
gRCNum = len(np.genfromtxt("/content/output/UNITRC.csv", delimiter=','))
gHNum = len(np.genfromtxt("/content/output/UNIHMDA.csv", delimiter=','))
gRNum = len(np.genfromtxt("/content/output/UNITRE.csv", delimiter=','))
gCarNum = len(np.genfromtxt("/content/output/CARGAS.csv", delimiter=','))

result = {
    "time_periods" : np.genfromtxt('/content/output/HORIZOMDA.csv', delimiter=',')[0],
    "demand": docSumByColum("PRODEM"),
    "reserves": docSumByColum("RRESUS")
}

In [ ]:
gen = {}
for i in range(gRCNum):
  tmp = {
    "must_run": 0,
    "name": "G{}".format(str(i+1))
  }
  tmp["power_output_minimum"] = extractFromFile(generator=i,file="LIUNITRC",method="min")
  tmp["power_output_maximum"] = extractFromFile(generator=i,file="LSUNITRC",method="max")
  tmp["ramp_up_limit"] = extractFromFile(generator=i,file="RAMPASRC",col = 0)
  tmp["ramp_down_limit"] = extractFromFile(generator=i,file="RAMPASRC",col = 1)
  tmp["ramp_startup_limit"] = extractFromFile(generator=i,file="ARRARC",col = 3)
  tmp["ramp_shutdown_limit"] = extractFromFile(generator=i,file="RAMPASRC",col = 1)
  tmp["time_up_minimum"] = int(extractFromFile(generator=i,file="OPPARORC",col = 1))
  tmp["time_down_minimum"] = int(extractFromFile(generator=i,file="OPPARORC",col = 0))
  tmp["power_output_t0"] = extractFromFile(generator=i,file="UNITRCCI",col = 2)
  tmp["unit_on_t0"] = 0 if extractFromFile(generator=i,file="UNITRCCI",col = 0) == 0 else 1
  tmp["time_up_t0"] = 0 if extractFromFile(generator=i,file="UNITRCCI",col = 0) == 0 else 1
  tmp["time_down_t0"] = 1 if tmp["unit_on_t0"] == 0 else 0
  tmp["fixed_cost"] = extractFromFile(generator=i,file="CGMRC", method = "max")
  lag,cost = extractFromFileByRange(i,"COVAARRC",7)
  tmp_lc = []
  for j,k in zip(lag,cost):
    tmp_lc.append({"lag":int(j),"cost":k})
  mw,_ = extractFromFileByRange(i,"POTVERC",11)
  cst,_ = extractFromFileByRange(i,"PREVERC",11)
  mw[0] = mw[0] + tmp["power_output_minimum"]
  tmp_mc = []
  for j,k in enumerate(zip(mw,cst)):
    tmp_mc.append({"mw":j,"cost":k[1]})
  tmp["startup"] = tmp_lc
  tmp["piecewise_production"] = tmp_mc
  gen["G"+str(i+1)] = tmp
result["thermal_generators"] = gen

NameError: ignored

In [ ]:
gen = {}
for i in range(gHNum):
  tmp = {
    "must_run": 0,
    "name": "H{}".format(str(i+1))
  }
  tmp["power_output_minimum"] = extractFromFile(generator=i,file="LIUNIH",method="min")
  tmp["power_output_maximum"] = extractFromFile(generator=i,file="LSUNIH",method="max")
  tmp["ramp_up_limit"] = extractFromFile(generator=i,file="RAMPASH",col = 0)
  tmp["ramp_down_limit"] = extractFromFile(generator=i,file="RAMPASH",col = 1)
  tmp["ramp_startup_limit"] = 0
  tmp["ramp_shutdown_limit"] = 0
  tmp["time_up_minimum"] = 0
  tmp["time_down_minimum"] = 0
  tmp["power_output_t0"] = 0
  tmp["unit_on_t0"] = 0 if extractFromFile(generator=i,file="UNIHCI",col = 0) == 0 else 1
  tmp["unit_up_t0"] = 0 if extractFromFile(generator=i,file="UNIHCI",col = 0) == 0 else 1
  tmp["time_down_t0"] = 1 if tmp["unit_on_t0"] == 0 else 0
  tmp["startup"] = [{"lag":0,"cost":0}]
  tmp["piecewise_production"] = [{"mw":0,"cost":0}]
  gen["H"+str(i+1)] = tmp
result["hydro_generators"] = gen

In [ ]:
gen = {}
for i in range(gRNum):
  tmp = {
    "must_run": 0,
    "name": "E{}".format(str(i+1))
  }
  tmp["power_output_minimum"] = extractFromFile(generator=i,file="LIUNITRE",method="min")
  tmp["power_output_maximum"] = extractFromFile(generator=i,file="LIUNITRE",method="max")
  mw,_ = extractFromFileByRange(i,"POTVERE",11)
  cst,_ = extractFromFileByRange(i,"PREVERE",11)
  mw[0] = mw[0] + tmp["power_output_minimum"]
  tmp_mc = []
  for j,k in enumerate(zip(mw,cst)):
    tmp_mc.append({"mw":sum(mw[:j]),"cost":k[1]})
  tmp["piecewise_production"] = tmp_mc
  gen["E"+str(i+1)] = tmp
result["renewable_generators"] = gen

In [ ]:
gen = {}
for i in range(gCarNum):
  tmp = {
    "must_run": 0,
    "name": "l{}".format(str(i+1))
  }
  tmp["fixed_load"] = extractFromFile(generator=i,file="CARFIJA",method="col",col=0)
  mw,_ = extractFromFileByRange(i,"POTCOCAR",11)
  cst,_ = extractFromFileByRange(i,"PRECOCAR",11)
  mw[0] = mw[0] + tmp["fixed_load"]
  tmp_mc = []
  for j,k in enumerate(zip(mw,cst)):
    tmp_mc.append({"mw":sum(mw[:j]),"cost":k[1]})
  tmp["piecewise_production"] = tmp_mc
  gen["l"+str(i+1)] = tmp
result["loads"] = gen

##Final

In [8]:
from os import listdir
from os.path import isfile, join
import numpy as np

class Generators:
  def __init__(self,path,catalogo):
    self.path = path
    self.cat = catalogo

  def validateFiles(self):
    returnable = True
    f = open("{}".format(self.cat),"r")
    validFiles = [i.replace("\n","") for i in f]
    f.close()
    onlyfiles = [f.split(".")[0] for f in listdir("{}".format(self.path)) if isfile(join("{}".format(self.path), f))]
    for i in validFiles:
      if i not in onlyfiles:
        print(i)
        returnable = False
    return returnable

  def docSumByColum(self,fileName):
    data = np.transpose(np.genfromtxt('{}/{}.csv'.format(self.path,fileName), delimiter=','))
    return [sum(i) for i in data][:-1]

  def extractFromFile(self,generator,file,method="col",col=None):
    if method == "min":
      return min(file[generator])
    if method == "max":
      return max(file[generator])
    if method == "col":
      return file[generator][col]  

  def extractFromFileByRange(self,generator,file,rang,min_val=0,pos1=0,pos2=1,method="mw"):
    lag = [min_val]
    cost = [0]
    for i,j in enumerate(file[generator*rang:(generator*rang)+rang]):
      if j[pos1] != 0 and j[pos2] != 0:
        if i == 0:
          lag[0] = min_val
          cost[0] = j[pos2]
        else:
          if method == "lag":
            lag.append(j[pos1])
          else:
            lag.append(lag[-1]+j[pos1])
          cost.append(j[pos2])
    return lag,cost

  def start(self):
    if self.validateFiles() is False:
      return "Faltan archivos"

    gRCNum = len(np.genfromtxt("{}/UNITRC.csv".format(self.path), delimiter=','))
    gHNum = len(np.genfromtxt("{}/UNIHMDA.csv".format(self.path), delimiter=','))
    gRNum = len(np.genfromtxt("{}/UNITRE.csv".format(self.path), delimiter=','))
    gCarNum = len(np.genfromtxt("{}/CARGAS.csv".format(self.path), delimiter=','))

    result = {
        "time_periods" : int(np.genfromtxt('{}/HORIZOMDA.csv'.format(self.path), delimiter=',')[0]),
        "demand": self.docSumByColum("PRODEM"),
        "reserves": self.docSumByColum("RRESUS")
    }
    gen = {}
    print("Generando datos de generadores termicos")
    LIUNITRC = np.genfromtxt("{}/LIUNITRC.csv".format(self.path), delimiter=',')
    LSUNITRC = np.genfromtxt("{}/LSUNITRC.csv".format(self.path), delimiter=',')
    RAMPASRC = np.genfromtxt("{}/RAMPASRC.csv".format(self.path), delimiter=',')
    ARRARC = np.genfromtxt("{}/ARRARC.csv".format(self.path), delimiter=',')
    OPPARORC = np.genfromtxt("{}/OPPARORC.csv".format(self.path), delimiter=',')
    UNITRCCI = np.genfromtxt("{}/UNITRCCI.csv".format(self.path), delimiter=',')
    CGMRC = np.genfromtxt("{}/CGMRC.csv".format(self.path), delimiter=',')
    COVAARRC = np.genfromtxt("{}/COVAARRC.csv".format(self.path), delimiter=',')
    POTVERC = np.genfromtxt("{}/POTVERC.csv".format(self.path), delimiter=',')
    PREVERC = np.genfromtxt("{}/PREVERC.csv".format(self.path), delimiter=',')
    for i in range(gRCNum):
      tmp = {
        "must_run": 0,
        "name": "G{}".format(str(i+1))
      }
      tmp["power_output_minimum"] = self.extractFromFile(generator=i,file=LIUNITRC,method="min")
      tmp["power_output_maximum"] = self.extractFromFile(generator=i,file=LSUNITRC,method="max")
      tmp["ramp_up_limit"] = self.extractFromFile(generator=i,file=RAMPASRC,col = 0)
      tmp["ramp_down_limit"] = self.extractFromFile(generator=i,file=RAMPASRC,col = 1)
      tmp["ramp_startup_limit"] = self.extractFromFile(generator=i,file=ARRARC,col = 3)
      tmp["ramp_shutdown_limit"] = self.extractFromFile(generator=i,file=RAMPASRC,col = 1)
      tmp["time_up_minimum"] = int(self.extractFromFile(generator=i,file=OPPARORC,col = 1))
      tmp["time_down_minimum"] = int(self.extractFromFile(generator=i,file=OPPARORC,col = 0))
      tmp["power_output_t0"] = self.extractFromFile(generator=i,file=UNITRCCI,col = 2)
      tmp["unit_on_t0"] = 0 if self.extractFromFile(generator=i,file=UNITRCCI,col = 0) == 0 else 1
      tmp["time_up_t0"] = 0 if self.extractFromFile(generator=i,file=UNITRCCI,col = 0) == 0 else 1
      tmp["time_down_t0"] = 1 if tmp["unit_on_t0"] == 0 else 0
      tmp["fixed_cost"] = self.extractFromFile(generator=i,file=CGMRC, method = "max")
      lag,cost = self.extractFromFileByRange(i,COVAARRC,7,min_val=1, method = "lag")
      tmp_lc = []
      for j,k in zip(lag,cost):
        tmp_lc.append({"lag":j,"cost":k})
      mw,_ = self.extractFromFileByRange(i,POTVERC,11,min_val=tmp["power_output_minimum"])
      cst,_ = self.extractFromFileByRange(i,PREVERC,11)
      tmp_mc = []
      
      if len(mw)==1:
          tmp_mc.append({"mw":self.extractFromFile(generator=i,file=LIUNITRC,method="min"),"cost":cst})
          tmp_mc.append({"mw":self.extractFromFile(generator=i,file=LSUNITRC,method="max"),"cost":cst})
      else:  
        for j,k in zip(mw,cst):
          tmp_mc.append({"mw":j,"cost":k})   

      tmp["startup"] = tmp_lc
      tmp["piecewise_production"] = tmp_mc
      gen["G"+str(i+1)] = tmp
    result["thermal_generators"] = gen
    del LIUNITRC,LSUNITRC,RAMPASRC,ARRARC,OPPARORC,UNITRCCI,CGMRC,COVAARRC,POTVERC,PREVERC
    print("Finalizado datos de generadores termicos")
    print("Generando datos de generadores hidraulicos")
    gen = {}
    LIUNIH = np.genfromtxt("{}/LIUNIH.csv".format(self.path), delimiter=',')
    LSUNIH = np.genfromtxt("{}/LSUNIH.csv".format(self.path), delimiter=',')
    RAMPASH = np.genfromtxt("{}/RAMPASH.csv".format(self.path), delimiter=',')
    UNIHCI = np.genfromtxt("{}/UNIHCI.csv".format(self.path), delimiter=',')
    for i in range(gHNum):
      tmp = {
        "must_run": 0,
        "name": "H{}".format(str(i+1))
      }
      tmp["power_output_minimum"] = self.extractFromFile(generator=i,file=LIUNIH,method="min")
      tmp["power_output_maximum"] = self.extractFromFile(generator=i,file=LSUNIH,method="max")
      tmp["ramp_up_limit"] = self.extractFromFile(generator=i,file=RAMPASH,col = 0)
      tmp["ramp_down_limit"] = self.extractFromFile(generator=i,file=RAMPASH,col = 1)
      tmp["ramp_startup_limit"] = 0
      tmp["ramp_shutdown_limit"] = 0
      tmp["time_up_minimum"] = 0
      tmp["time_down_minimum"] = 0
      tmp["power_output_t0"] = 0
      tmp["unit_on_t0"] = 0 if self.extractFromFile(generator=i,file=UNIHCI,col = 0) == 0 else 1
      tmp["unit_up_t0"] = 0 if self.extractFromFile(generator=i,file=UNIHCI,col = 0) == 0 else 1
      tmp["time_down_t0"] = 1 if tmp["unit_on_t0"] == 0 else 0
      tmp["startup"] = [{"lag":1,"cost":0}]
      tmp["piecewise_production"] = [{"mw":0,"cost":0}]
      gen["H"+str(i+1)] = tmp
    result["hydro_generators"] = gen
    del LIUNIH,LSUNIH,RAMPASH,UNIHCI
    print("Finalizado datos de generadores hidraulicos")
    print("Generando datos de generadores renovables")
    LIUNITRE = np.genfromtxt("{}/LIUNITRE.csv".format(self.path), delimiter=',')
    POTVERE = np.genfromtxt("{}/POTVERE.csv".format(self.path), delimiter=',')
    PREVERE = np.genfromtxt("{}/PREVERE.csv".format(self.path), delimiter=',')
    gen = {}
    for i in range(gRNum):
      tmp = {
        "must_run": 0,
        "name": "E{}".format(str(i+1))
      }
      tmp["power_output_minimum"] = self.extractFromFile(generator=i,file=LIUNITRE,method="min")
      tmp["power_output_maximum"] = self.extractFromFile(generator=i,file=LIUNITRE,method="max")
      mw,_ = self.extractFromFileByRange(i,POTVERE,11,min_val=tmp["power_output_minimum"])
      cst,_ = self.extractFromFileByRange(i,PREVERE,11)
      tmp_mc = []
      for j,k in zip(mw,cst):
        tmp_mc.append({"mw":j,"cost":k})
      tmp["piecewise_production"] = tmp_mc
      gen["E"+str(i+1)] = tmp
    result["renewable_generators"] = gen
    del LIUNITRE,POTVERE,PREVERE
    gen = {}
    print("Finalizado datos de generadores renovables")
    print("Generando datos de cargas")
    CARFIJA = np.genfromtxt("{}/CARFIJA.csv".format(self.path), delimiter=',')
    POTCOCAR = np.genfromtxt("{}/POTCOCAR.csv".format(self.path), delimiter=',')
    PRECOCAR = np.genfromtxt("{}/PRECOCAR.csv".format(self.path), delimiter=',')
    for i in range(gCarNum):
      tmp = {
        "must_run": 0,
        "name": "l{}".format(str(i+1))
      }
      tmp["fixed_load"] = self.extractFromFile(generator=i,file=CARFIJA,method="col",col=0)
      mw,_ = self.extractFromFileByRange(i,POTCOCAR,11)
      cst,_ = self.extractFromFileByRange(i,PRECOCAR,11)
      tmp_mc = []
      for j,k in zip(mw,cst):
        tmp_mc.append({"mw":j,"cost":k})
      tmp["piecewise_production"] = tmp_mc
      gen["l"+str(i+1)] = tmp
    result["loads"] = gen
    del CARFIJA,POTCOCAR,PRECOCAR
    print("Finalizado datos de cargas")
    f = open("{}/output.json".format(self.path), "w")
    f.write(str(result).replace("\'","\""))
    f.close()
    return "Archivo {}/output.json creado".format(self.path)

In [9]:
g = Generators("/content/dirdat","/content/catalogo.txt")
g.start()

Generando datos de generadores termicos
Finalizado datos de generadores termicos
Generando datos de generadores hidraulicos
Finalizado datos de generadores hidraulicos
Generando datos de generadores renovables
Finalizado datos de generadores renovables
Generando datos de cargas
Finalizado datos de cargas


'Archivo /content/dirdat/output.json creado'